#### Pre- Sum D

In [152]:
# ! pip install pdfplumber
import pdfplumber
import pandas as pd
import numpy as np

#**
file= r"0.pdf"
volinj= 100
injrep_cal=[3]*6 #'70','85','100','115','130','qcs'
samrep= 3
bracrep= samrep
sumpage = [0]


with pdfplumber.open(file) as pdf:
    df_0=[]
    for i in sumpage:
        page = pdf.pages[i].extract_table()
        # b= [
        #     [row[i] for i in [i for i, col in enumerate(list(zip(*page))) if col.count(None) < len(page)/2]]
        #     for row in page
        # ]
        df_0.extend(
            [
            [row[i] for i in [i for i, col in enumerate(list(zip(*page))) if col.count(None) < len(page)/2]]
            for row in page
        ]
        )
    df_0=pd.DataFrame(df_0)

df_0.iloc[30:36]

,0,1,2,3,4,5,6,7,8,9
30,18,85%-3,Y:A4,Calibration Standard,2,100.00,AMI-2288 Oxytocin,1,1.0000,Finished
31,19,100%,Y:A5,Calibration Standard,3,100.00,AMI-2288 Oxytocin,1,1.0000,Finished
32,20,100%,Y:A5,Calibration Standard,3,100.00,AMI-2288 Oxytocin,1,1.0000,Finished
33,21,100%-3,Y:A5,Calibration Standard,3,100.00,AMI-2288 Oxytocin,1,1.0000,Finished
34,22,115%,Y:A6,Calibration Standard,4,100.00,AMI-2288 Oxytocin,1,1.0000,Finished
35,23,115%,Y:A6,Calibration Standard,4,100.00,AMI-2288 Oxytocin,1,1.0000,Finished


In [153]:
for i in range(0,df_0.shape[1]):
    if df_0.iloc[:,i].str.contains('Finished', case=False, na=False).any():
        df_0.rename(columns={i:'Status'}, inplace= True)
    if df_0.iloc[:,i].str.contains('Unknown', case=False, na=False).any():
        df_0.rename(columns={i:'Type'}, inplace= True)
    if df_0.iloc[:,i].str.contains('100%', case=False, na=False).any():
        df_0.rename(columns={i:'Name1'}, inplace= True)
    if df_0.iloc[:,i].str.contains('shut|rinse', case=False, na=False).any():
        df_0.rename(columns={i:'AMI'}, inplace= True)
    if df_0.iloc[:,i].str.contains('10', case=False, na=False).any() & df_0.iloc[:,i].str.contains('12', case=False, na=False).any():
        df_0.rename(columns={i:'dm'}, inplace= True)
    if df_0.iloc[16:,i].str.contains(r'[a-zA-Z]').any():
        df_0.rename(columns={i:'Vial'}, inplace= True)
    if df_0.iloc[:,i].str.contains(str(volinj)).sum() > df_0.shape[0]/2:
        df_0.rename(columns={i:'Vol'}, inplace= True)
    if df_0.iloc[:,i].str.fullmatch(r'1').sum() > df_0.shape[0]/2:
        df_0.rename(columns={i:'Fill'}, inplace= True)
    if df_0.iloc[:,i].str.fullmatch(r'1').sum() >= injrep_cal[0]:
        df_0.rename(columns={i:'Level'}, inplace= True)
    else:
        df_0.rename(columns={i:'DF'}, inplace= True)
df_0.tail()

,dm,Name1,Vial,Type,Level,Vol,AMI,Fill,DF,Status
53,41,1181354-3,Y:B2,Unknown,,100.00,AMI-2288 Oxytocin,1,104.1667,Finished
54,42,100% Final Bracket,Y:A5,Unknown,,100.00,AMI-2288 Oxytocin,1,1.0000,Finished
55,43,100% Final Bracket,Y:A5,Unknown,,100.00,AMI-2288 Oxytocin,1,1.0000,Finished
56,44,100% Final Bracket-3,Y:A5,Unknown,,100.00,AMI-2288 Oxytocin,1,1.0000,Finished
57,45,discard shutdown,Y:A1,Unknown,,100.00,Quantiva Shutdown,1,1.0000,Finished


In [156]:
df1_0=df_0[df_0['Vial'].astype(str).str.contains(r'\d+$', regex= True)]
df1_0=df1_0[~df1_0.astype(str).apply(lambda row: row.str.contains('Discard|rinse', case=False, na=False)).any(axis=1)]
df1dc_0=df1_0[df1_0.astype(str).apply(lambda row: row.str.contains('discard|rinse', case=False, na=False)).any(axis=1)]

df1_0['Vol']=pd.to_numeric(df1_0['Vol'], errors='coerce')
df1_0['DF']=pd.to_numeric(df1_0['DF'], errors='coerce')
df1_0['Level']=pd.to_numeric(df1_0['Level'], errors='coerce')
vol=df1_0.groupby("Vol")['Vol'].mean()
if len(vol)!= 1 or vol.iloc[0]!= volinj:
    print(f'Wrong inj vol')
else:
    pass

df2=df1_0[df1_0["Status"] != "Finished"]
if df2.shape[0]!=0:
    print(f'not finished')
else:
    pass

cal=[]
for i in ['70%','85%','100%','115%','130%','qcs|check']:
    cal1= (~df1_0['Name1'].str.contains('bracket', case=False) & df1_0['Name1'].str.contains(i, case=False)).sum()
    cal.append(cal1)
if np.array_equal([int(i) for i in cal],injrep_cal) != True: 
    print(f'Ck calib replicate')
else:
    pass

sam= df1_0[df1_0['Name1'].str.contains(r'\b\d{7}\b', regex=True)]
sam['Name']= sam['Name1'].str.split('-', n=1).str[0]
sam1=sam.groupby('Name')['DF'].describe().iloc[:,:3]

notsam= df1_0[~df1_0['Name1'].str.contains(r'\b\d{7}\b', regex=True)]
notsam['Name1']= notsam['Name1'].str.split('-', n=1).str[0]
notsam1= notsam.groupby('Name1')['DF'].describe().iloc[:,:3]


df1_0['Name']= df1_0['Name1'].str.split('-', n=1).str[0]

brac1= df1_0[df1_0['Name'].str.contains('bracket', case=False)]
brac=brac1.groupby('Name')['Name'].describe().iloc[:,3]
if np.array_equal(np.array(brac), np.array([bracrep]*len(np.array(brac)))) != True: ###
    print(f'Ck bracket replicate')
else:
    pass

df1d= df1_0.groupby('Name')['DF'].describe().iloc[:,:3].reset_index()
print(f'DF of all\n-------\n {df1d[['Name','mean']]}') # count 6/2
if (df1d[df1d['std'] > 0.0000001]).shape[0] !=0:
    print(f'Check DF of\n{df1d[df1d['std'] > 0.00001]['Name']}')
else:
    pass

df1l= df1_0.groupby('Name')['Level'].describe().iloc[:,:3].reset_index()
if (df1l[df1l['std'] > 0.0000001]).shape[0] !=0:
    print(f'Check level assignment of\n{df1l[df1l['std'] > 0.00001]['Name']}')
else:
    pass
print(f'Level assignment\n-------\n{df1l[['Name','mean']].sort_values(by='mean')}')

df1v= df1_0.groupby('Name')['Vial'].describe().reset_index() #.iloc[:,:3]
df1v 
mask= (df1v['freq'] != samrep) | (df1v['unique'] != 1) 
if df1v[mask].any().any():
    print(f'Check vial pos of\n-------\n{(df1v[mask])['Name']}')
else:
    pass

df1vv= df1_0.groupby('Vial')['Name'].describe().reset_index()
if df1vv[df1vv['unique'] != 1].any().any():
    print(f'Check vial pos at\n-------\n{(df1vv[df1vv['unique'] != 1])['Vial']}')
else:
    pass
print(df1vv[df1vv['unique'] != 1]) #(except 100%)

samv= df1v[df1v['Name'].str.contains(r'\b\d{7}\b', regex=True)]['top']
df1dc_0['check']= df1dc_0['Vial'].str.contains(pd.Series(samv).iloc[0])
samvcheck= df1dc_0[df1dc_0['check']==True]
if samvcheck.shape[0]!=0:
    print(f'Sample in discard!')
else:
    pass

std= df1_0[df1_0['Type'].str.contains('Standard')]
df1_0['Level']=df1_0['Level'].replace(r'^\s*$',np.nan, regex=True).fillna(0)
lev= df1_0[df1_0['Level']!=0]
lev
if std.equals(lev)!=True:
    print(f'Non-std has level')
else:
    pass

DF of all
-------
                   Name      mean
0                    0    1.0000
1                   00    1.0000
2                 100%    1.0000
3   100% Final Bracket    1.0000
4                 115%    1.0000
5              1179015    0.6250
6              1181339  104.1667
7              1181354  104.1667
8                 130%    1.0000
9                  70%    1.0000
10                 85%    1.0000
11      Check Standard    1.0000
Level assignment
-------
                  Name  mean
9                  70%   1.0
10                 85%   2.0
2                 100%   3.0
4                 115%   4.0
8                 130%   5.0
11      Check Standard   6.0
0                    0   NaN
1                   00   NaN
3   100% Final Bracket   NaN
5              1179015   NaN
6              1181339   NaN
7              1181354   NaN
Check vial pos of
-------
0     0
1    00
Name: Name, dtype: object
Check vial pos at
-------
4    Y:A5
Name: Vial, dtype: object
   Vial count unique

C:\Users\hpham\AppData\Local\Temp\ipykernel_28864\3999875196.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sam['Name']= sam['Name1'].str.split('-', n=1).str[0]
C:\Users\hpham\AppData\Local\Temp\ipykernel_28864\3999875196.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notsam['Name1']= notsam['Name1'].str.split('-', n=1).str[0]


#### Calib Top D

In [157]:
# ! pip install pytesseract pillow
import pytesseract
pytesseract.pytesseract.tesseract_cmd= r'C:\Users\hpham\Downloads\tesseract.exe' #**

#**
calib_page =[1] #pdf-1
gh= ['Oxy','Amount','Area']

word_h= ["Lin,",'Slope',"Square","IS",'Amount',"Acetyl"]
word_v= ['Area'] 
reso= [3,5,7]

In [ ]:
# ! pip install easyocr
import fitz
import easyocr
from PIL import Image
import gc

### might need results from both reso

def xtttii(page, word_h= word_h, word_v= word_v, reso= reso): 
    text_hf, text_vf= [],[]
    ocrlen_hf, ocrlen_vf, outtext_hf=[], [], []

    for ii in reso:
    
        mat= fitz.Matrix(ii,ii) #even is shit, "I" nah at 3
        pix= page.get_pixmap(matrix=mat)
        img= Image.frombytes('RGB',[pix.width,pix.height], pix.samples) #RGB
        
        text= pytesseract.image_to_string(img)
        text= text.splitlines()
        text_h= [i for i in text if any(j in i for j in word_h)] 
        ocrlen_h= len(text_h[0])

        rot=img.rotate(-90, expand= True) #*   
        result= easyocr.Reader(['en']).readtext(np.array(rot))
        ba=[]
        for a,b,c in result: 
            ba.append(b.split())
        text1= [i for j in ba for i in j] 
        text_v= [i for i in text1 if any(j in i for j in word_v)] 
        if len(text_v)== 0:
            ocrlen_v= 0
        else:
            ocrlen_v= len(text_v[0])

        del pix
        gc.collect()
        
        text_hf.append(text_h), text_vf.append(text_v)
        ocrlen_hf.append(ocrlen_h), ocrlen_vf.append(ocrlen_v)

    for i in range(0,len(reso)):
        otext_hf= len(text_hf[i])
        outtext_hf.append(otext_hf)

    maxindocr= [i for i, val in enumerate(ocrlen_hf) if val>= max(ocrlen_hf)-4]
    maxindout= [i for i, val in enumerate(outtext_hf) if val== max(outtext_hf)] 
    besth= list(set(maxindocr) & set(maxindout))

    bestv= ocrlen_vf.index(max(ocrlen_vf))
    
    return text_hf[besth[0]], text_vf[bestv]
    

doc= fitz.open(file)
d=[]
for i in calib_page: 
    page= doc[i] 
    d.append(xtttii(page, word_h= word_h, word_v= word_v, reso= reso))
    del page
    gc.collect()
print(d)
doc.close() #1min for one

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
c:\Users\hpham\AppData\Local\miniconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


IndexError: list index out of range

In [ ]:
text= d[0]
def cal_page(gh, text):
    if len(text[1])!=0:
        ya= gh[2] in text[1][0]
        an = gh[0] in text[0][3]
        xa= gh[1] in text[0][4] 
    else:
        ya= 'no y-label'
        an = gh[0] in text[0][3]
        xa= 'no x-label'
    return an, xa, ya
an, xa, ya= cal_page(gh, text)
an, xa, ya

(False, True, True)

In [171]:
def regparams_pix(text, regp= 'Lin, WithOffset, Avg'):
    set= ('Lin, '+text[0][text[0].find('Lin') + len('Lin') + 2:][0:15]) == regp
    yint= float(text[0][text[0].find('(CO)') + len('(CO)') + 1:][0:15])
    slope= float(text[1][text[1].find('(C1)') + len('(C1)') + 1:][0:15])
    mse= float(text[2][text[2].find('R-Square') + len('R-Square') + 1:][0:6])
    return set, yint, slope, mse
set, yint, slope, mse = regparams_pix(text) #[0]
set, yint, slope, mse

ValueError: could not convert string to float: 'rument:HPLC#58 '

#### Sum

In [254]:
#** 3
header= ["Oxy","IS","Roc","Ace"]
Std_conc, QCS_conc= 2.172, 2.172
Volinit_std, Volinit_qcs=[4,5,6,7,3], 6
Volser_std, Volser_qcs=[1]*5, 0.52128
Volfin_std, Volfin_qcs=[125, 125, 125, 125, 50], 125
samcon= ['1179015', '1181339', '1181354'] 
fillvol= [542,1,1]
lbclaim= [30,10,10] ## make sure in this order

compage=[2]

In [255]:
with pdfplumber.open(file) as pdf:
    df0=[]
    for i in compage:
        p= pdf.pages[i].extract_tables()
        for x in p:
            df00= pd.DataFrame(x)
            df0.append(df00)
df= pd.concat(df0, ignore_index=True)
# pd.unique(df[[8,9]].values.ravel()) 
df=df.iloc[:,:8]
# df.head()

In [256]:
sum_i= df[df.iloc[:,1].astype(str).str.contains('injection', case=False)].index ###
sum_i
# len(df_1)>0

Index([8], dtype='int64')

In [257]:
df=df.iloc[sum_i[0]:]
df.head()

,0,1,2,3,4,5,6,7
8,No.,Injection Name,Ret.Time,Ret. Time,Area,Area,Ratio,Fill_Volume
9,,,min,min,counts*min,counts*min,,
10,,,Oxytocin,Oxytocin IS,Oxytocin,Oxytocin IS,Oxytocin,
11,1,Discard,1.84,1.83,5,54010,0.0001,1
12,2,Discard extraction chk,1.63,1.62,69080,41609,1.6602,1


In [258]:
df.columns=['No','Name1','RT','RTIS','Area','AreaIS','Ratio','Amount'] #**
df=df.reset_index(drop=True)
df1=df[~df.astype(str).apply(lambda row: row.str.contains('discard|rinse', case=False, na=False)).any(axis=1)]
df1=df1.set_index("No")
# df1.head(5)

In [259]:
df2= df1.copy()
term= header
df3={}
for i in term:
    count= df2.apply(lambda col: col.str.contains(i, case=False).sum())
    df3[i]= count
df3 = pd.DataFrame(df3)
df3

,Oxy,IS,Roc,Ace
Name1,0,0,0,0
RT,1,0,0,0
RTIS,1,1,0,0
Area,1,0,0,0
AreaIS,1,1,0,0
Ratio,1,0,0,0
Amount,0,0,0,0


In [260]:
df4=df1[1:].iloc[:,:-1].applymap(lambda cell: str(cell).replace(",",""))
for i in range(1,6):
    df4.iloc[:,i]=pd.to_numeric(df4.iloc[:,i], errors='coerce')
# df4[45:55]

C:\Users\hpham\AppData\Local\Temp\ipykernel_28864\36289412.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df4=df1[1:].iloc[:,:-1].applymap(lambda cell: str(cell).replace(",",""))


In [261]:
### output the one that are not
df4r=df4[~df4["Name1"].str.contains('0|00', case=False, na=False)]
df4r["dRT"]= (df4r["RT"]-df4r["RTIS"]).abs()
df5=df4r[df4r["dRT"]>0.2]
df5 
if len(df5)!=0:
    print("Is it RocBro?")
else:
    pass

C:\Users\hpham\AppData\Local\Temp\ipykernel_28864\3253397052.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4r["dRT"]= (df4r["RT"]-df4r["RTIS"]).abs()


In [ ]:
### RT shift
### RSD IS

In [262]:
df4na=df4[df4.isna().any(axis=1)]
print(df4na)

   Name1   RT RTIS Area AreaIS Ratio
No                                  
          NaN  NaN  NaN    NaN   NaN
          NaN  NaN  NaN    NaN   NaN


In [263]:
df4["Name"]= df4['Name1'].str.split('-', n=1).str[0]
df6=df4.groupby(["Name"])["Ratio"].mean()
df6= pd.DataFrame(df6).reset_index().sort_values(by='Ratio')
print(f'Analyte/IS ratio\n-------\n{df6}')
df7= (df4.groupby(["Name"])["Ratio"].std()/df4.groupby(["Name"])["Ratio"].mean()*100).sort_values()
df7  

Analyte/IS ratio
-------
                  Name     Ratio
1                    0   0.00015
2                   00   0.06445
10                 70%     1.287
11                 85%  1.598267
6              1179015  1.635133
4   100% Final Bracket  1.939633
3                 100%  1.947267
12      Check Standard    1.9511
5                 115%     2.304
9                 130%  2.501033
7              1181339  2.694833
8              1181354     3.049
0                            NaN


Name
115%                   0.132147
85%                    0.202292
1181354                0.492762
100% Final Bracket     0.598259
130%                   0.705992
100%                   0.759872
1179015                0.760916
1181339                0.837988
70%                    0.930975
Check Standard          1.01423
00                     6.692554
0                     47.140452
                            NaN
Name: Ratio, dtype: object

In [ ]:
df6.iloc[0,1]/df6[df6['Name']=="130%"].iat[0,1] >0.01 ###

In [247]:
arrx

array([0.069504, 0.069504, 0.069504, 0.08688 , 0.08688 , 0.08688 ,
       0.104256, 0.104256, 0.104256, 0.121632, 0.121632, 0.121632,
       0.13032 , 0.13032 , 0.13032 ])

In [264]:
from scipy.stats import linregress

df9=[]
calc1= QCS_conc * Volinit_qcs* Volser_qcs / Volfin_qcs
for i in range(0,len(Volinit_std)):
    calc= Std_conc * Volinit_std[i] * Volser_std[i] / Volfin_std[i]
    df9.append(calc)
df9.append(calc1)    

call=['70%','85%','115%','130%','100%']
pattern='|'.join(call)
df4l=df4[df4["Name1"].str.contains(pattern, case=False, na=False) & 
         ~df4["Name1"].str.contains('bracket', case=False, na=False)] #.iloc[:30,:]
# df4ll= pd.merge(levcal['Name1'],df4l, on='Name1', how= 'left').drop_duplicates(subset="Area")
arry= df4l["Ratio"]

arrx=np.repeat(np.array([df9[:5]]), injrep_cal[:5]) 

slope_c, int, r, p, stderr = linregress(arrx.tolist(), arry.tolist())
print(f'slope: {slope_c} \ny-intercept: {int} \nR^2: {r**2}')

slope: 19.981583793738498 
y-intercept: -0.12096666666666733 
R^2: 0.9982945833329021


In [83]:
abs((int-yint))/yint * 100 < 0.4, abs((slope_c-slope))/slope * 100 < 0.2, abs((r**2-mse))/mse * 100 < 0.7

(np.True_, np.True_, np.True_)

In [265]:
df6["Theo_conc"]= (df6["Ratio"]- int)/slope_c
df9=pd.DataFrame(df9)
df9["Conc"]=['70%','85%','100%','115%','130%','QCS'] #**
df9.columns=['Exp_amount','Name']
df9a= pd.DataFrame({"Name":['100% Bracket1','100% Final Bracket'],
                    "Exp_amount": [df9.iloc[2,0], df9.iloc[2,0]]}) #**
df9=pd.concat([df9,df9a],axis=0,ignore_index=True)
df10= pd.merge(df6,df9, on='Name', how= 'right')
df10["%_recovery"]= df10['Theo_conc']/df10['Exp_amount']*100 ###
df10

,Name,Ratio,Theo_conc,Exp_amount,%_recovery
0,70%,1.287,0.070463,0.069504,101.380088
1,85%,1.598267,0.086041,0.086880,99.034178
2,100%,1.947267,0.103507,0.104256,99.281554
3,115%,2.304,0.12136,0.121632,99.776443
4,130%,2.501033,0.131221,0.130320,100.691244
5,QCS,NaN,NaN,0.054347,NaN
6,100% Bracket1,NaN,NaN,0.104256,NaN
7,100% Final Bracket,1.939633,0.103125,0.104256,98.915131


In [267]:
sam_1= df6[df6['Name'].str.contains('|'.join(samcon))].drop(columns='Ratio') 
sam_1
mask= df1_0['Name1'].str.contains('|'.join(samcon)) #& df_0['Name1'].str.contains('L') #, df_0['Name'].str.contains(samcon1)

df_1= pd.concat([df1_0[sam_1]],axis=0) 
df1_0['DF']=pd.to_numeric(df1_0['DF'], errors='coerce')
dfad= df1_0.groupby('Name')['DF'].mean()
df11= pd.merge(sam_1,dfad, on='Name', how= 'left')
df11['Fill_vol']= fillvol
df11["Calc_amount"]= df11['Theo_conc'] * df11['DF'] * df11['Fill_vol']
df11['Claim']= lbclaim 
df11['PC']=(df11["Calc_amount"]/df11['Claim']*100).astype(float).round(2)
df11


,Name,Theo_conc,DF,Fill_vol,Calc_amount,Claim,PC
0,1179015,0.087886,0.6250,542,29.771358,30,99.24
1,1181339,0.14092,104.1667,1,14.679146,10,146.79
2,1181354,0.158644,104.1667,1,16.525465,10,165.25


#### Calib Bot D

In [86]:
with pdfplumber.open(file) as pdf:
    df=[]
    for i in calib_page:
        p= pdf.pages[i].extract_table()
        df.extend(
            [
            [row[i] for i in [i for i, col in enumerate(list(zip(*p))) if col.count(None) < len(p)/2]]
            for row in p
        ]
        )
        
df_cal= pd.DataFrame(df)
# df_cal=df_cal.loc[:,~df_cal.loc[6].isna()]
# df_cal.columns= range(df_cal.shape[1])
df_cal.tail()

,0,1,2,3,4,5,6
67,41,130%,5,3.2479,1.4220,"1,507,007","11,857,239"
68,42,130%,5,3.2479,1.3756,"1,503,960","11,800,933"
69,43,130%,5,3.2479,1.3672,"1,532,413","12,088,255"
70,44,130%,5,3.2479,1.3710,"1,502,383","11,875,788"
71,45,130%-6,5,3.2479,1.3530,"1,527,469","12,073,345"


In [87]:
df_cal=df_cal[df_cal.iloc[:,3].astype(str).str.contains(r'\d+$', regex= True)].drop(df_cal.columns[[0,5,6]], axis=1) #**
df_cal
all([np.array_equal(np.array(df_cal.iloc[:,0]), np.array(df4l['Name1'])),
        np.allclose(np.array(pd.to_numeric(df_cal.iloc[:,2], errors='coerce')),np.array(pd.to_numeric(arrx, errors='coerce')), rtol=0.0001),
        np.allclose(np.array(pd.to_numeric(df_cal.iloc[:,3], errors='coerce')),np.array(pd.to_numeric(arry, errors='coerce')), rtol=0.0001)])
# all([i for i in check])

True

#### w

In [98]:
with pdfplumber.open(file) as pdf:
    df=[]
    for i in range (3,23):
        p= pdf.pages[i].extract_table()
        df.extend(p)

df_cal_sst= pd.DataFrame(df)
### prev regex doesnt work .iloc[:,8]

df_cal_sst= (df_cal_sst[df_cal_sst.iloc[:,8].astype(str)
                        .str.contains(r'-?\d+(\.\d+)?([eE][-+]?\d+)?', regex= True)]
                        .drop(df_cal_sst.columns[[0]], axis=1)
)

C:\Users\hpham\AppData\Local\Temp\ipykernel_27896\110817157.py:11: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  .str.contains(r'-?\d+(\.\d+)?([eE][-+]?\d+)?', regex= True)]


In [100]:
df_cal_sst.iloc[:,3]=pd.to_numeric(df_cal_sst.iloc[:,3], errors='coerce')
df_cal_sst.iloc[:,4]=pd.to_numeric(df_cal_sst.iloc[:,4], errors='coerce')
df_cal_sst.iloc[:,5]=pd.to_numeric(df_cal_sst.iloc[:,5], errors='coerce')

In [101]:
df_cal_sst_ns= df_cal_sst[~df_cal_sst.astype(str).apply(lambda row: row.str.contains(r'\b\d{7}\b', regex=True)).any(axis=1)]
df_cal_sst_ns[:60]

,1,2,3,4,5,6,7,8
2,Glycine,RSD of SSuit 70%,<=,3.5,NaN,0.4114,Passed,70%
3,Glycine,,,NaN,NaN,NaN,,70%
4,Glycine,,,NaN,NaN,NaN,,70%-3
5,Glycine,Average of SSuit 70%,between,95.0,105.0,99.5937,Passed,70%
6,Glycine,,,NaN,NaN,NaN,,70%
7,Glycine,,,NaN,NaN,NaN,,70%-3
11,Glycine,RSD of SSuit 85%,<=,3.5,NaN,0.1423,Passed,85%
12,Glycine,,,NaN,NaN,NaN,,85%
13,Glycine,,,NaN,NaN,NaN,,85%-3
14,Glycine,Average of SSuit 85%,between,95.0,105.0,98.9803,Passed,85%


In [74]:
df_cal_sst_ns.describe().iloc[:3,]

,1,2,3,4,5,6,7,8
count,80,80,80,80,80,80,80,80
unique,1,20,3,3,3,20,2,19
top,Glycine,,,,,,,100%


In [81]:
all(df_cal_sst_ns.iloc[:,2].unique() == np.array(['<=', '', 'between']))

True

In [102]:
[round(df_cal_sst_ns.iloc[:,3].unique(), 2) if isinstance(i,(int,float)) else i for i in df_cal_sst_ns.iloc[:,3]]

TypeError: isinstance() arg 2 must be a type, a tuple of types, or a union

In [ ]:
# !pip install tabula-py
# !pip install JPype1
# import tabula
# df= tabula.read_pdf(file,pages='2')
# df
# Failed to import jpype dependencies. Fallback to subprocess.
# No module named 'jpype'
# No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly


#### T

In [272]:
import fitz
import easyocr
from PIL import Image
import gc

word_h= ["Lin,",'Slope',"Square","IS",'Amount',"Acetyl"]
word_v= ['Area'] 
gh= ['Oxy','Amount','Area'] #*
reso= [3,5,7]

doc= fitz.open(file)
page= doc[1] #*

text_hf, text_vf= [],[]
ocrlen_hf, ocrlen_vf, outtext_hf=[], [], []

for ii in reso:
    
    mat= fitz.Matrix(ii,ii) #even is shit, "I" nah at 3
    pix= page.get_pixmap(matrix=mat)
    img= Image.frombytes('RGB',[pix.width,pix.height], pix.samples) #RGB
    
    text= pytesseract.image_to_string(img)
    text= text.splitlines()
    text_h= [i for i in text if any(j in i for j in word_h)] 
    ocrlen_h= len(text_h[0]) #[0]
    

    rot=img.rotate(-90, expand= True) #*   
    result= easyocr.Reader(['en']).readtext(np.array(rot))
    ba=[]
    for a,b,c in result: 
        ba.append(b.split())
    text1= [i for j in ba for i in j] 
    text_v= [i for i in text1 if any(j in i for j in word_v)] 
    if len(text_v)== 0:
        ocrlen_v= 0
    else:
        ocrlen_v= len(text_v[0]) #[0]

    del pix
    gc.collect()
    
    text_hf.append(text_h), text_vf.append(text_v)
    ocrlen_hf.append(ocrlen_h), ocrlen_vf.append(ocrlen_v)


text_hf, text_vf, ocrlen_hf, ocrlen_vf

for i in range(0,len(reso)):
    otext_hf= len(text_hf[i])
    outtext_hf.append(otext_hf)

maxindocr= [i for i, val in enumerate(ocrlen_hf) if val>= max(ocrlen_hf)-4]
maxindout= [i for i, val in enumerate(outtext_hf) if val== max(outtext_hf)] 
besth= list(set(maxindocr) & set(maxindout))

bestv= ocrlen_vf.index(max(ocrlen_vf))
besth, bestv
text_hf[besth[0]], text_vf[bestv]

# RuntimeError: [enforce fail at alloc_cpu.cpp:116] data. DefaultCPUAllocator: not enough memory: you tried to allocate 1300234240 bytes.
# MemoryError: Unable to allocate 51.0 MiB for an array with shape (1856, 2400, 3) and data type float32

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
c:\Users\hpham\AppData\Local\miniconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


(['Calibration Type Lin, WithOffset, Avg Offset (CO) -0.1210',
  'Evaluation Type Area Slope (C1) 19.9819',
  'Number of disabled Calibration Points _0 R-Square 1.00',
  '300- Oxytocin Internal ‘Oxytocin IS’ MS Quantitation',
  '% ISTD'],
 [])

In [ ]:
# df_0=np.vstack([df_000.values,df_00.values])


In [ ]:
# with pdfplumber.open(file) as pdf:
#     df_000=[]
#     for i in range(0,1):
#         page = pdf.pages[i].extract_table()
#         # df_000.extend(page)
#     # df_000= pd.DataFrame(df_000[2:])
#     # df_000= df_000.drop(columns= df_000.columns[df_000.isna().sum() > df_000.shape[0]/2])
# # df_00= [[0 if i is None else i for i in inner] for inner in df_000]

# df_00= list(zip(*page))
# coll= [i for i, col in enumerate(df_00) if col.count(None) < len(page)/2]
# a= [
#     [row[i] for i in coll]
#     for row in page
# ]
# # a= pd.DataFrame(a)
# a
# # df_00

In [ ]:
# with pdfplumber.open(file) as pdf:
#     df_000=[]
#     for i in range(0,2):
#         page = pdf.pages[i].extract_table()
#         df_000.extend(page)
#     df_000= pd.DataFrame(df_000[2:])
#     # df_000= df_000.drop(columns= df_000.columns[df_000.isna().sum() > df_000.shape[0]/2])
# df_000.tail()

In [ ]:
# with pdfplumber.open(file) as pdf:
#     df=[]
#     for i in sumpage:
#         p= pdf.pages[i].extract_table()
#         df.extend(p)

# df= pd.DataFrame(df) 

In [ ]:
# ocrlen_h, ocrlen_v=[], []
# for i in [0,len(reso)-1]:
#     ocrlen_h.append(len(text_hf[i][0]))
#     ocrlen_v.append(len(text_vf[i][0]))
# ocrlen_h, ocrlen_v
# besth, bestv= ocrlen_h.index(max(ocrlen_h)), ocrlen_v.index(max(ocrlen_v))
# besth, bestv

In [ ]:
#w too comp

# with pdfplumber.open(file) as pdf:
#     df=[]
#     for i in calib_page:
#         p= pdf.pages[i].extract_table()
#         df.extend(p)

# df_cal= pd.DataFrame(df)
# # df_cal=df_cal.loc[:,~df_cal.loc[6].isna()]
# # df_cal.columns= range(df_cal.shape[1])
# df_cal[0:8]

# line= df_cal.iloc[6,0]

# def regparams_l(line):
#     set= 'Lin, '+line[line.find('Lin') + len('Lin') + 2:][0:15]
#     yint= float(line[line.find('Offset (C0)') + len('Offset (C0)') + 1:][0:6])
#     slope= float(line[line.find('Slope (C1)') + len('Slope (C1)') + 1:][0:6])
#     mse= float(line[line.find('R-Square') + len('R-Square') + 1:][0:5])
#     return set, yint, slope, mse

# search= ['Lin','Offset','Slope','R-Square']

# def regparams_t(search):
#     tab= []
#     for i in search:
#         mask= df_cal.stack()[df_cal.stack().str.contains(i, na=False)]
#         for(x,y), value in mask.items():
#             tab.append({
#                 'word': i,
#                 'Set': value,
#                 'xref': x,
#                 'yref':y
#                 })
#     tab= pd.DataFrame(tab)
#     set= tab.iloc[0,1]+df_cal.iloc[tab.iloc[0,2],tab.iloc[0,3]+1]
#     # df_cal.iloc[tab.iloc[0,2],tab.iloc[0,3]+1])
#     yint= float(df_cal.iloc[tab.iloc[1,2],tab.iloc[1,3]+1])
#     slope= float(df_cal.iloc[tab.iloc[2,2],tab.iloc[2,3]+1])
#     mse= float(df_cal.iloc[tab.iloc[3,2],tab.iloc[3,3]+1])
#     return set, yint, slope, mse


# if len(line) >20:
#     set, yint, slope, mse= regparams_l(line)
# else:
#     set, yint, slope, mse= regparams_t(search)


In [ ]:
# fitz same output as plumber

In [ ]:
# !pip install "camelot-py[cv]" work well for the bottom
# import camelot
# df= camelot.read_pdf(file, pages = '5')
# df[0].df.iloc[2,0]
#'Calibration Type\nLin, WithOffset, Avg\nOffset (C0)\n0.2479\nEvaluation Type\nArea\nSlope (C1)\n0.1604'
#'Calibration Type\nOffset (C0)\nLin, WithOffset, Avg\n0.0163\nEvaluation Type\nSlope (C1)\nArea\n0.2294\nNumber of Calibration Points\nCurve (C2)\n5\n0\nNumber of disabled Calibration Points\nR-Square\n0\n1.00'

In [ ]:
# w,h= img.size
# top= img.crop((0,0, w,h //2))

# pix= fitz.Rect(0,0,page.rect.width, page.rect.width /2)

In [ ]:
# result= easyocr.Reader(['en']).readtext('1.png')
# ba=[]
# for a,b,c in result: 
#     ba.append(b.split())
# ba
# text1= [i for j in ba for i in j]

# words= ['Area']
# text2= [i for i in text1 if any(j in i for j in words)] 
# text2

In [ ]:
# text= text.splitlines()
# words= ['Internal','Amount']
# text3= [i for i in text if any(j in i for j in words)] 
# text3

In [ ]:
# text1= text.split()
# ind= text1.index('Internal')
# aft, bef= text1[ind+1:ind+3], text1[ind-5:ind-1]
# print(aft, bef)

In [ ]:
# import re

# picword= ['500']
# search= re.search(rf'\b{picword}\b\s+(\w+)', text,re.IGNORECASE)
# if search:
#     waft=search.group(1)
#     print(f'After: {waft}')
# else:
#     pass
# so: words list arent really one after another

# print(text[230:280],text[300:350])

In [ ]:
# import fitz #pymupdf
# from PIL import Image
# import io

# doc= fitz.open("58ereport_2.pdf")
# page= doc[2]
# imag= page.get_images()

# def ocr_rot(page):
#     mat= fitz.Matrix(5,5)
#     pix= page.get_pixmap(matrix=mat)
#     # img= Image.open(io.BytesIO(pix.tobytes('png')))
#     img= Image.frombytes('RGB',[pix.width,pix.height], pix.samples)
#     result=[]
#     for ang in [0,90,180, 270]:
#         rot=img.rotate(ang, expand= True)
#         text= pytesseract.image_to_string(rot)
#         result.append((ang,text.strip()))
#         print(result)
# ocr_rot(page)

# for i in [3,5,7,9]: #3 maybe 5
#     mat= fitz.Matrix(i,i)
#     pix= page.get_pixmap(matrix=mat)
#     pix.save('1.png')
#     img= Image.frombytes('RGB',[pix.width,pix.height], pix.samples)
#     text= pytesseract.image_to_string(img)
#     print(text)

# split into many images
# for iind, img in enumerate(imag):
#     ref=img[0]
#     baseimg= doc.extract_image(ref)
#     byt= baseimg['image']
#     ext= baseimg['ext']
#     name= f'img{iind + 1}.{ext}'
#     with open(name, 'wb') as f:
#         f.write(byt)
#     print(f'save as {name}')

In [ ]:
# df_0.columns=['dm','Name1','Vial','Level','Type','Vol','AMI','DF','Status'] #** 
# df_0.columns=['dm','Name1','Vial','Type','Level','Vol','DF','Status'] #** 
# df_0.columns=['dm','Type','Vial','Name1','Level','Vol','AMI','DF'] #**
# df_0.columns=['dm','Level','Type','Name1','AMI','Vial','Vol','DF','Status'] #**
# df_0.iloc[:,7].replace(r'^\s*$',np.nan, regex=True).isna().sum()